<a href="https://colab.research.google.com/github/Andrea-24744/Simulaci-n-1/blob/main/Inventarios_Ej55.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### **Ejercicio 5.5. Sistema de Inventarios**
*Andrea Santelices Medina*

La demanda mensual de cierto producto sigue la siguiente distribución de probabilidad  empírica:

| Cantidad | Probabilidad |
|----------|--------------|
|   35     |    0.010     |
|   36     |    0.015     |
|   37     |    0.020     |
|   38     |    0.020     |
|   39     |    0.022     |
|   40     |    0.023     |
|   41     |    0.025     |
|   42     |    0.027     |
|   43     |    0.028     |
|   44     |    0.029     |
|   45     |    0.035     |
|   46     |    0.045     |
|   47     |    0.060     |
|   48     |    0.065     |
|   49     |    0.070     |
|   50     |    0.080     |
|   51     |    0.075     |
|   52     |    0.070     |
|   53     |    0.065     |
|   54     |    0.060     |
|   55     |    0.050     |
|   56     |    0.040     |
|   57     |    0.030     |
|   58     |    0.016     |
|   59     |    0.015     |
|   60     |    0.005     |

---
El tiempo de entrega está distribuido de la siguiente manera:

|  Meses      | Probabilidad |
|-------------|--------------|
| 1           | 0.3          |
| 2           | 0.4          |
| 3           | 0.3          |

---

Los factores estacionales para cada uno de los meses del año son como se muestra a continuación:

| Mes | Factores estacionales | Mes | Factores estacionales |
|-----|------------------------|-----|------------------------|
|  1  | 1.20                   |  7  | 0.80                   |
|  2  | 1.00                   |  8  | 0.90                   |
|  3  | 0.90                   |  9  | 1.00                   |
|  4  | 0.80                   | 10  | 1.20                   |
|  5  | 0.80                   | 11  | 1.30                   |
|  6  | 0.70                   | 12  | 1.40                   |

---

La información con respecto a los costos relevantes es la siguiente:

- **Costo de ordenar** = \$100 por orden  
- **Costo de inventario** = \$20 por unidad por año  
- **Costo de faltante** = \$50 por unidad  

---

Si el inventario inicial se asume en 150 unidades, ¿determine la cantidad óptima a ordenar $q$ y el nivel óptimo de reorden $ R $?

---

*Este ejercicio está resuelto en el libro de forma analítica, empleando el método de la transformada inversa, ahora se resolverá en base a la simulación:*


In [15]:
import numpy as np

# Definimos nuetra primera tabla de probabilidades de la demanda mensual, de manera que vamos a ir sumando las probabilidades
# (es decir, definiremos la función de distribución acumulada: pdf )
demanda_cdf = np.array([0.01, 0.025, 0.045, 0.065, 0.087, 0.11, 0.135, 0.162,
                     0.19, 0.219, 0.254, 0.299, 0.359, 0.424, 0.494, 0.574,
                     0.649, 0.719, 0.784, 0.844, 0.894, 0.934, 0.964, 0.98, 0.995,1])

# Establecemos las cantidades demandadas, segun la tabla 1
demanda_vals = np.arange(35, 61)

# Hacemos una funcion que asigne el valor correspondiente segun el intervalo en el que caiga u
# (posteriormente le meteremos un random para la aleatoriedad)
def gen_demanda(u):
  i=0
  while u > demanda_cdf[i]:
    i+=1
  return demanda_vals[i]


# Nuestra segunda tabla
def tiempo_entrega(u):
  if u < 0.3:
    return 1
  elif u < 0.7:
    return 2
  else:
    return 3

# Nuestra tercer tabla
factores = np.array([1.2, 1, 0.9, 0.8, 0.8, 0.7, 0.8, 0.9, 1, 1.2, 1.3, 1.4])
meses = 24  # Simularemos 2 años
costos = {'orden': 100, 'almacen': 20/12, 'faltante': 50}  # Nuestra información de cstos

In [16]:
import numpy as np
import random
from random import random

def simular(r, q):
  inventario = 150 # Inventario inicial (150 unidades)
  pedidos_pendientes = []  # lista de fechas de entrega
  costos_totales = 0
  for mes in range(meses):
    pedidos_pendientes = [m for m in pedidos_pendientes if m >= mes] # Si la fecha de llegada es mayor al mes actual, está pendiente
    for m in pedidos_pendientes:
      if m == mes: # Si la fecha de llegada es el mes actual...
          inventario += q # Llega el pedido
    pedidos_pendientes = [m for m in pedidos_pendientes if m > mes] # Actualizamos la lista de pedidos pendientes

    # Calculamos la demanda del mes
    demanda = gen_demanda(random())*factores[mes%12] # La multiplicamos por el factor estacional

    if demanda <= inventario: # Si tenemos suficientes artículos en inventario
      inventario -= demanda # Surtimos la demanda
      faltante = 0
    else: # Si no tenemos suficientes artículos
      faltante = demanda - inventario # Surtimos lo que podemos
      inventario = 0

    # ¿Hay que hacer un pedido?
    if inventario <= r: # Si tenemos menos articulos que el nivel de reorden, pedimos más
      entrega = tiempo_entrega(random())
      mes_llegada = mes + entrega
      pedidos_pendientes.append(mes_llegada) # Añadimos el pedido a la lista de pedidos pendientes
      costos_totales += costos['orden'] # Sumamos el costo del pedido

    costos_totales += inventario * costos['almacen']
    costos_totales += faltante * costos['faltante'] # Callculamos los demás costos al final del mes

  return costos_totales*12/meses # Regresa el costo anual

def buscar_rq():
    mejor = []
    for r in range(35, 200):        # Niveles de reorden
      for q in range(35, 200):    # Cantidades a pedir
        costos_simulacion = [simular(r, q) for _ in range(20)] # 20 Simulaciones (de 2 años c/u) para cada par (r,q)
        promedio = np.mean(costos_simulacion)
        mejor.append((promedio, r, q)) # Guardamos los costos promedio, junto a su correspindoente par (r,q)

    mejor = min(mejor) # Nos quedamos con el par (r,q) que nos genere el menor costo
    return mejor

# Ejecutar
optimos = buscar_rq()
print('El nivel de reorden óptimo es: %d\nY la cantidad a ordenar óptima es: %d\n con un costo anual esperado de: %.2f'%(optimos[1],optimos[2],optimos[0]))

El nivel de reorden óptimo es: 169
Y la cantidad a ordenar óptima es: 49
 con un costo anual esperado de: 2725.35
